In [1]:
!pip install torch torchvision scikit-learn matplotlib


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Load Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# One-hot encode the target labels
encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y.reshape(-1, 1))

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [4]:
class IrisDataset(torch.utils.data.Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]


train_dataset = IrisDataset(X_train, y_train)
test_dataset = IrisDataset(X_test, y_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)

In [11]:
[x for x in train_loader]

[[tensor([[-0.0525, -0.8226,  0.7628,  0.9223],
          [-1.5065,  0.7888, -1.3402, -1.1838],
          [-0.2948, -0.5924,  0.6491,  1.0539],
          [ 0.1898, -0.1320,  0.5922,  0.7907],
          [-0.2948, -0.8226,  0.2512,  0.1325],
          [ 0.4322,  0.7888,  0.9333,  1.4488],
          [ 1.0380,  0.5586,  1.1038,  1.1856],
          [-0.0525, -0.5924,  0.7628,  1.5805],
          [ 1.0380,  0.0982,  1.0469,  1.5805],
          [ 1.0380, -0.1320,  0.7059,  0.6590],
          [ 1.6438,  1.2492,  1.3311,  1.7121],
          [ 0.1898,  0.7888,  0.4217,  0.5274],
          [ 0.1898, -0.8226,  0.7628,  0.5274],
          [-0.1737, -0.3622,  0.2512,  0.1325],
          [ 1.4015,  0.3284,  0.5354,  0.2641],
          [ 1.5227, -0.1320,  1.2175,  1.1856]]),
  tensor([[0., 0., 1.],
          [1., 0., 0.],
          [0., 0., 1.],
          [0., 0., 1.],
          [0., 1., 0.],
          [0., 0., 1.],
          [0., 0., 1.],
          [0., 0., 1.],
          [0., 0., 1.],
          [0.,

In [5]:
class IrisNN(nn.Module):
    def __init__(self):
        super(IrisNN, self).__init__()
        self.fc1 = nn.Linear(4, 64)  # Input layer to hidden layer
        self.fc2 = nn.Linear(64, 32)  # Hidden layer to hidden layer
        self.fc3 = nn.Linear(32, 3)  # Hidden layer to output layer

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = IrisNN()

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
def train(model, train_loader, criterion, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, torch.argmax(target, dim=1))
        loss.backward()

        # Gradient Analysis
        for name, param in model.named_parameters():
            if param.grad is not None:
                print(f"{name} gradient norm: {torch.norm(param.grad).item()}")

        optimizer.step()


# Train the model for one epoch
train(model, train_loader, criterion, optimizer)

fc1.weight gradient norm: 0.14965187013149261
fc1.bias gradient norm: 0.06694688647985458
fc2.weight gradient norm: 0.6040263772010803
fc2.bias gradient norm: 0.11281225085258484
fc3.weight gradient norm: 0.5184771418571472
fc3.bias gradient norm: 0.15994761884212494
fc1.weight gradient norm: 0.17518223822116852
fc1.bias gradient norm: 0.0578448660671711
fc2.weight gradient norm: 0.7059093117713928
fc2.bias gradient norm: 0.08849722892045975
fc3.weight gradient norm: 0.5640684366226196
fc3.bias gradient norm: 0.23635351657867432
fc1.weight gradient norm: 0.12029307335615158
fc1.bias gradient norm: 0.042090147733688354
fc2.weight gradient norm: 0.4528151750564575
fc2.bias gradient norm: 0.06935412436723709
fc3.weight gradient norm: 0.373281866312027
fc3.bias gradient norm: 0.14339140057563782
fc1.weight gradient norm: 0.09646125882863998
fc1.bias gradient norm: 0.051008306443691254
fc2.weight gradient norm: 0.40905827283859253
fc2.bias gradient norm: 0.09415452182292938
fc3.weight gradi

In [8]:
def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            _, predicted = torch.max(output, 1)
            total += target.size(0)
            correct += (predicted == torch.argmax(target, dim=1)).sum().item()
    accuracy = correct / total
    print(f"Test Accuracy: {accuracy * 100:.2f}%")


evaluate(model, test_loader)

Test Accuracy: 66.67%
